# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.127014e+02     1.645968e+00
 * time: 0.3686330318450928
     1     1.016898e+01     9.779359e-01
 * time: 2.30930495262146
     2    -1.274010e+01     1.060000e+00
 * time: 2.9797821044921875
     3    -3.424456e+01     9.063216e-01
 * time: 3.9657609462738037
     4    -4.784768e+01     6.961946e-01
 * time: 4.9306581020355225
     5    -5.684164e+01     2.601375e-01
 * time: 5.850058078765869
     6    -5.965884e+01     2.301717e-01
 * time: 6.458002090454102
     7    -6.082223e+01     7.272457e-02
 * time: 7.060070991516113
     8    -6.131443e+01     7.199065e-02
 * time: 7.695044040679932
     9    -6.159552e+01     7.445760e-02
 * time: 8.311662912368774
    10    -6.182426e+01     3.323727e-02
 * time: 8.909193992614746
    11    -6.199562e+01     2.444121e-02
 * time: 9.509629011154175
    12    -6.208239e+01     1.602704e-02
 * time: 10.084341049194336
    13    -6.213631e+01     1.305628e-02
 * time: 10.656018018722534
  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671073
    AtomicLocal         -18.8557662
    AtomicNonlocal      14.8522624
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485374 
    Xc                  -19.3336822

    total               -62.261666461071
